The story of this interlude is the story of a battle lost (or at best drawn) against technocentrism. This interlude was started with the intention of coming up with the simplest possible Python example in which a reader could explore what they like about their creation and try out automatic music generation. What it became was a code sample in which a beat is generated that is reasonably interesting, quite easy to explain, and endlessly fiddlable. Is it: a trap; a starting point; a generative worked example; or are we just justifying interesting code with rhetoric? (This is always a danger.)

1. fictionalize relationship 
1. how might a kid try this? what issues would they run into?
1. what might a kid not know?
1. democratizing data can be expressing oneself.

Perhaps fitting for the last interlude in this book that we *do not* start with the importing of libraries, exactly, but, rather the installation of new software. In this case, sound libraries are required. Do not get too caught up in why this is not in Python by default – it often is, but this specific platform is not typically intended for music. For now, ignore the streams of nonsense-looking output that may result from software installation. These are noteworthy considerations around data use and reproduction, but are beyond the scope of the lesson offered here.


In [ ]:
!apt install fluidsynth # this is the library that the computer needs to generate sound
!git clone https://github.com/nwhitehead/pyfluidsynth # this connects it to Python
!python ./pyfluidsynth/setup.py install # this installs the downloaded python connnector

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  fluid-soundfont-gm libfluidsynth1 libqt5x11extras5 qsynth
Suggested packages:
  fluid-soundfont-gs timidity jackd
The following NEW packages will be installed:
  fluid-soundfont-gm fluidsynth libfluidsynth1 libqt5x11extras5 qsynth
0 upgraded, 5 newly installed, 0 to remove and 41 not upgraded.
Need to get 120 MB of archives.
After this operation, 150 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 fluid-soundfont-gm all 3.1-5.1 [119 MB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libfluidsynth1 amd64 1.1.9-1 [137 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic/universe amd64 fluidsynth amd64 1.1.9-1 [20.7 kB]
Get:4 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libqt5x11extras5 amd64 5.9.5-0ubuntu1 [8,596 B]
Get:5 http://archive.ubuntu.com/ubuntu bionic/uni

Now we get to import our libraries!

In [ ]:
from IPython.display import Audio # this lets us play music in the cell
from pyfluidsynth import fluidsynth # this is the Python connector we installed above
import numpy as np # our data analysis library
from random import random, randint, choice # random numbers


Contained below is the entirety of our music generation software. It is short! You can do this! 

There are arguments in a couple directions here: this might seem daunting, confusing, or exclusionary; it could just as easily feel exploratory, freeing, or revelatory. Creative work can often be like that. Questions remain – as throughout the book: What might we expect you to know or not know? What might we expect a student to know or not know? How does code like this interplay with identity? To what extent is this likely to feel like it is off-putting and to what extent is this an entry to seeing that anyone can do creative work like this. Confidence - perhaps self-efficacy - is needed, but the musical knowledge required is almost nothing. For example, you might look at the code below and decide that you do not know why E Minor consists of those numbers. I (Berland) have no idea why E Minor consists of those numbers, but that does not bother me; I know relatively little about music theory, so I just looked it up. Knowing that I can just look it up – and knowing that and knowing who I could ask for help – is more of the creative block here than "not knowing" but resources and power are inequitably distributed. 

In short, make some music by fiddling with the numbers below. It really si fun, and it would be surprising if there was not some music made with some instrument accessible from the code below was not pleasing to every single person who could read this. Who knows?

In [ ]:
fl = fluidsynth.Synth()
fl.start()

# the arcane location just below is where the sound software put the 
# "sound fonts" - that is, the libraries of instrument sounds by note
sfid = fl.sfload("/usr/share/sounds/sf2/FluidR3_GM.sf2") 

# you can mess with this! change the key!

piano_e_minor_notes = [0, 64, 66, 67, 69, 71, 72, 74]

# this is just "blank" and "bass drum" but many other drums exist!
drum_bassdrum_notes = [0, 47]

# 4 on the flour. 16th notes
seqlen = 16


# track ("program") 0 is drums
fl.program_select(0, sfid, 128, 0)

# track 1 is piano
fl.program_select(1, sfid, 0, 0)

s = np.array([])
seconds = 0.07 # how long is a 16th note?


# each of these just selects from the lists above
# either silence or the notes we put in that list
drum_riff =  [choice(drum_bassdrum_notes) for _ in range(seqlen)]
piano_riff = [choice(piano_e_minor_notes) for _ in range(seqlen)]

for i in range(seqlen * 4): # repeat the riff 4 times so it sounds riffy
  
  drum_note = drum_riff[i % len(drum_riff)] # play the right note
  # the "%" means "mod" - mod is "integer remainder" - like, 4 % 3 = 1
  # we do that so we can go around and around a list of the right size 
  if 0 != drum_note: # if not silence
    fl.noteon(0,drum_note,20)

  piano_note = piano_riff[i%len(piano_riff)] # same idea as the drums
  if 0 != piano_note:
    fl.noteon(1,piano_note,20)
  

  s = np.append(s, fl.get_samples(int(44100 * seconds))) 
  # just tack the notes on the end

fl.delete() # ok, let's play!

framerate = 44100 # this is what our player expects. like a CD
Audio(s, rate=framerate, autoplay=True) # and let us hear the riff